In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
import pickle
import datetime
import os

In [2]:
# global variables for data pathfiles

FDNY_RAW = "raw_data/Incidents_Responded_to_by_Fire_Companies.csv"
NYC_ZIPS = 'raw_data/NYC_ZIPS/ZIP_CODE_040114.shp'
PLUTO_BK = 'raw_data/PLUTO/Brooklyn/BKMapPLUTO.shp'
PLUTO_BX = 'raw_data/PLUTO/Bronx/BXMapPLUTO.shp'
PLUTO_MN = 'raw_data/PLUTO/Manhattan/MNMapPLUTO.shp'
PLUTO_QN = 'raw_data/PLUTO/Queens/QNMapPLUTO.shp'
PLUTO_SI = 'raw_data/PLUTO/Staten_Island/SIMapPLUTO.shp'
MASTER_PLUTO_PICKLE = 'processed_data/master_pluto.pickle'
DOB_COMPLAINTS = 'raw_data/DOB_Complaints_Received.csv'
DOB_ECB = 'raw_data/DOB_ECB_Violations.csv'
DOB_VIOLATIONS = 'raw_data/DOB_Violations.csv'
DOB_PERMITS = 'raw_data/Historical_DOB_Permit_Issuance.csv'
PAD = 'raw_data/PAD/bobaadr.txt'
CENSUS_TRACT_RACE = 'raw_data/CENSUS_TRACT_RACE_INCOME/ACS_15_5YR_DP05_with_ann.csv'
CENSUS_TRACT_INCOME = 'raw_data/CENSUS_TRACT_RACE_INCOME/ACS_15_5YR_S1901_with_ann.csv'
ZIP_TRACT = 'raw_data/zip_tract_122015.csv'

### Import raw data

FDNY data

In [3]:
# import FDNY data
fdny = pd.read_csv(FDNY_RAW,usecols=['IM_INCIDENT_KEY',
                                     'INCIDENT_TYPE_DESC','ZIP_CODE'],
                   dtype={'ZIP_CODE':str})

NYC zip code and census tract shapefiles

In [4]:
# import NYC zipcode shapefiles
nyc_zips = gp.read_file(NYC_ZIPS)

# read in zip_tract
zip_tract = pd.read_csv(ZIP_TRACT,usecols=['ZIP','TRACT'],dtype={'ZIP':str,'TRACT':str})
zip_tract.rename(columns = {'ZIP':'ZIPCODE'},inplace=True)

# merge to nyc zips
zip_tract_nyc = nyc_zips.merge(zip_tract,how='left',on='ZIPCODE')

NYC PLUTO (2015)

In [5]:
def import_filter_pluto():
    
    # import PLUTO for 5 boros
    BK = gp.read_file(PLUTO_BK)
    BX = gp.read_file(PLUTO_BX)
    MN = gp.read_file(PLUTO_MN)
    QN = gp.read_file(PLUTO_QN)
    SI = gp.read_file(PLUTO_SI)
    
    # merge 5 boro PLUTO datasets 
    pluto_agg = BK.append(BX)
    pluto_agg = pluto_agg.append(MN)
    pluto_agg = pluto_agg.append(QN)
    pluto_agg = pluto_agg.append(SI)
    
    # select key columns

    pluto_select = pluto_agg[['ZipCode',
    'BldgClass',
    'LandUse',
    'BldgArea',
    'ComArea',
    'ResArea',
    'OfficeArea',
    'RetailArea',
    'UnitsRes',
    'UnitsTotal',
    'AssessTot',
    'YearBuilt',
    'BuiltFAR','LotArea']]
    
    # create pickle

    with open(MASTER_PLUTO_PICKLE, 'wb') as handle:
        pickle.dump(pluto_select, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
# ***********************
# NOTE: Since Geopandas does not allow filtering select columns, 
# you'll have to load full PLUTO set, merge, and select columns,
# then save as a pickle file for later use.
# ***********************

if os.path.exists(MASTER_PLUTO_PICKLE):
    print "File exists. Loading pickle..."
    # load pickle of PLUTO data
    with open(MASTER_PLUTO_PICKLE, 'rb') as handle:
        master_pluto = pickle.load(handle)
    
else:
    print "File does not yet exist. Importing and filtering PLUTO. This could take several minutes..."
    # first time only, import, filter, and save processed PLUTO as a pickle for future use
    import_filter_pluto()
    
    # load pickle of PLUTO data
    with open(MASTER_PLUTO_PICKLE, 'rb') as handle:
        master_pluto = pickle.load(handle)

File exists. Loading pickle...


DOB and ECB permits and violations

In [7]:
# DOB complaints
dob_complaints = pd.read_csv(DOB_COMPLAINTS,usecols=['Complaint Number', 'Date Entered', 
                                 'BIN', 'Complaint Category', 
                                 'Disposition Date','Disposition Code', 
                                 'Inspection Date'])

In [8]:
# DOB violations 
dob_violations = pd.read_csv(DOB_VIOLATIONS,dtype={'BORO':str,
                                                   'BLOCK':str,'LOT':str,
                                                   'ISSUE_DATE':str,
                                                   'DISPOSITION_DATE':str})

In [9]:
# ECB violations
ecb = pd.read_csv(DOB_ECB,usecols=['BIN','BORO','BLOCK','LOT','SEVERITY','VIOLATION_TYPE',
                                   'VIOLATION_DESCRIPTION',
                                   'INFRACTION_CODE1','ISSUE_DATE',
                                   'SECTION_LAW_DESCRIPTION1'],dtype={'BORO':str,
                                                                      'BLOCK':str,
                                                                      'LOT':str,
                                                                      'ISSUE_DATE':str})

/Users/Daynan/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
# DOB work permits
permits = pd.read_csv(DOB_PERMITS,usecols=['Zip Code','BOROUGH','Block','Lot',
                                           'Bldg Type','Residential','Permit Type',
                                           'Oil Gas','Issuance Date'],dtype={'BOROUGH':str,
                                                                           'Block':str,
                                                                           'Lot':str})

Census Data

In [11]:
# race data
race = pd.read_csv(CENSUS_TRACT_RACE,skiprows=[1],usecols=['GEO.id2', 'HC01_VC03', 
                                                           'HC01_VC49', 'HC01_VC50', 
                                                           'HC01_VC51','HC01_VC56', 
                                                           'HC01_VC64', 'HC01_VC69', 'HC01_VC23'])

# rename columns
race.rename(columns={'HC01_VC03': 'TOTAL_POPULATION', 'HC01_VC49': 'WHITE',
                        'HC01_VC50': 'BLACK_AFRICAN_AMERICAN', 'HC01_VC51': 'AMERICAN_INDIAN_AND_ALASKA_NATIVE',
                        'HC01_VC56': 'ASIAN', 'HC01_VC64': 'NATIVE_HAWAIIAN_AND_OTHER_PACIFIC_ISLANDER',
                        'HC01_VC69': 'SOME_OTHER_RACE', 'HC01_VC23': 'MEDIAN_AGE', 'GEO.id2': 'GEOID'}, inplace=True)

# convert values to float
def make_float(expected_float):
    try:
        if type(expected_float) == str:
            expected_float = expected_float.replace('+', '').replace(',', '')
        return float(expected_float)
    except:
        # print expected_float
        return np.nan

for i in race.columns[race.columns!='GEOID']:
    race[i] = race[i].apply(lambda x: make_float(x))

In [12]:
# income data
income = pd.read_csv(CENSUS_TRACT_INCOME,skiprows=[1],usecols=['GEO.id2', 'HC01_EST_VC01', 'HC01_EST_VC15'])

# rename columns
income.rename(columns={'HC01_EST_VC01': 'TOTAL_HOUSEHOLDS', 
                       'HC01_EST_VC15': 'MEAN_INCOME', 'GEO.id2': 'GEOID'}, inplace=True)

#convert values to float
for i in income.columns[income.columns!='GEOID']:
    income[i] = income[i].apply(lambda x: make_float(x))

### Filter, aggregate and scale data

- Filter FDNY by gas leaks
- Aggregate FDNY by zip codes
- Preprocess and scale PLUTO attributes
- Aggregate PLUTO data by zip
- Preprocess and scale building data attributes
- Aggregate building data by zip 
- Aggregate census tract data by zip

#### Filter FDNY for gas leaks and aggregate by zip

In [13]:
# split incident description to get code, filter
def code_split(data):
    a = data.split(' -')
    return a[0]

fdny['incident_code'] = fdny.INCIDENT_TYPE_DESC.apply(lambda x: code_split(x))
fdny_gas = fdny[fdny.incident_code=='412']

# clean FDNY zip data and aggregate by zip
fdny_gas_zip = pd.DataFrame(fdny_gas.groupby('ZIP_CODE')[
        'IM_INCIDENT_KEY'].count()).reset_index()

#### Preprocess and scale PLUTO attributes

In [14]:
# convert PLUTO zip to string
master_pluto['ZipCode'] = master_pluto['ZipCode'].astype(str)

In [15]:
# calculate building age
def year_calc(data):
    if (data < 1800) | (data > 2016):
        return float('NaN')
    else:
        return 2017-data

master_pluto['age'] = master_pluto.YearBuilt.apply(lambda x: year_calc(x))

In [16]:
def scale_and_group_zip(data,zip_col_name,field,multiple=True,
                        header_prefix=None,dispose=False,og=False):
    '''Creates a general "group by" and scaling function that:
    - groups data for a given variable category in an input dataframe by zip code
    - then creates a ratio of each variable category in the zip for all values in zip
    - produces a new sparse matrix with rows=zip codes and cols=each category of the variable,
    where the values are the ratio of the category / all instances in the zip
    '''
    # group selected variable by zip code
    if multiple:
        if og:
            data2 = data.copy()
            data2[field] = data2[field].fillna('NA')
            data = data2.copy()
        
        temp_df = data.groupby([zip_col_name,field])[
            field].count().unstack(level=-1).reset_index()
        
        # create df of ratio of select category of variable per all instances in zip
        zip_matrix = pd.DataFrame()

        for i in range(len(temp_df[zip_col_name])):
            zip_matrix[str(
                temp_df[zip_col_name][i])] = temp_df.T[i][1:]/temp_df.T[i][1:].sum()
    
        zip_matrix = zip_matrix.T.reset_index()

        zip_matrix['ZipCode'] = zip_matrix['index'].astype(str)
        if og:
            zip_matrix = zip_matrix.drop(['index','NA'],axis=1)
        else:
            zip_matrix = zip_matrix.drop('index',axis=1)
        
        if dispose:
            zip_matrix = zip_matrix.rename(columns={zip_matrix.columns[0]:'No_disposition'})
            
        # catch decimal zipcodes and strip
        if len(zip_matrix.ZipCode[10])>5:
            zip_matrix['ZipCode'] = zip_matrix['ZipCode'].apply(lambda x: x[:-2])

        
        # update header to specific source data (for less confusion when merging data later)
        if header_prefix:
            new_columns = []
            for col in zip_matrix.columns:
                if col != 'ZipCode':
                    new_columns.append(header_prefix+col)
                else:
                    new_columns.append(col)
            zip_matrix.columns = new_columns
            
        return zip_matrix       
    
    else:
    
        return pd.DataFrame((data.groupby(zip_col_name)[field].sum())/data.groupby(
                zip_col_name)[field].count()).reset_index()

In [17]:
# groupby zip - average age
avg_bldg_age_by_zip = scale_and_group_zip(master_pluto,'ZipCode','age',multiple=False)

In [18]:
# groupby zip - # building class in zip / total building in zip
bldgclass_by_zip = scale_and_group_zip(master_pluto,'ZipCode','BldgClass',
                                       multiple=True,header_prefix='bldg_class_')

In [19]:
# groupby zip - # landuse in zip / total land uses in zip
landuse_by_zip = scale_and_group_zip(master_pluto,'ZipCode','LandUse',
                                       multiple=True,header_prefix='landuse_')

In [20]:
# the following assorted pluto attributes are aggregated by zip
pluto_attrib_by_zip = pd.DataFrame()

# function to create ratio of given PLUTO category per zip code
def pluto_attributes_zip(data,zip_col_name,oldfield,newfield,denominator='BldgArea'):
    pluto_attrib_by_zip[newfield] = data.groupby(
        zip_col_name)[oldfield].sum()*1.0/data.groupby(zip_col_name)[denominator].sum()

In [21]:
# commercial ratio by zip code
pluto_attributes_zip(master_pluto,'ZipCode','ComArea','com_ratio',denominator='BldgArea')

# residential ratio by zip code
pluto_attributes_zip(master_pluto,'ZipCode','ResArea','res_ratio',denominator='BldgArea')

# office ratio by zip code
pluto_attributes_zip(master_pluto,'ZipCode','OfficeArea','office_ratio',denominator='BldgArea')

# retail ratio by zip code
pluto_attributes_zip(master_pluto,'ZipCode','RetailArea','retail_ratio',denominator='BldgArea')

# res / total units by zip code
pluto_attributes_zip(master_pluto,'ZipCode','UnitsRes','res_unit_ratio',denominator='UnitsTotal')

# mean unit area by zip code
pluto_attributes_zip(master_pluto,'ZipCode','BldgArea','unit_area',denominator='UnitsTotal')

# assessed value per sq foot
pluto_attributes_zip(master_pluto,'ZipCode','AssessTot','value_per_ft',denominator='LotArea')

In [22]:
# total units by zip code
pluto_attrib_by_zip['total_units'] = master_pluto.groupby('ZipCode')['UnitsTotal'].sum()

# reset index
pluto_attrib_by_zip = pluto_attrib_by_zip.reset_index()

#### Preprocess and scale the DOB and ECB data

In [23]:
# merge BIN with zipcodes pulled from another dataset
pad = pd.read_csv(PAD,usecols=['bin','zipcode'])
# zip to int
def zipint(data):
    try:
        return str(int(data))
    except ValueError:
        return float('NaN')
pad['zipcode'] = pad.zipcode.apply(lambda x: zipint(x))
pad = pad.rename(columns={'bin':'BIN'})
pad = pad[~pad.zipcode.isnull()]

In [24]:
# merge zip to dob/ecb datasets
dob_complaints = dob_complaints.merge(pad,how='left',on='BIN')
dob_violations = dob_violations.merge(pad,how='left',on='BIN')
ecb = ecb.merge(pad,how='left',on='BIN')

Slice DOB data to include only data before 12/31/2015, same as FDNY data for now, we'll consider issuances before 1/1/2013 to be relevant.

In [25]:
# convert date string to datetime
dob_complaints['date_entered'] = dob_complaints['Date Entered'].apply(
    lambda x: datetime.datetime.strptime(x,'%m/%d/%Y'))

In [26]:
permits['issuance_date'] = permits['Issuance Date'].apply(
    lambda x: datetime.datetime.strptime(x,'%m/%d/%Y'))

In [27]:
# deal with inconsistent datetime
def dob_date(data):
    try:
        return datetime.datetime.strptime(data, '%Y%m%d')
    except:
        try:
            y,md = data.split('  ')
            if y in ['11','12','13','14','15']:
                data = '20'+y+md
                return datetime.datetime.strptime(data, '%Y%m%d')
        except:
            try:
                data = str(data)[:8]
                return datetime.datetime.strptime(data, '%Y%m%d')
            except:
                return float("NaN")

dob_violations['issue_date'] = dob_violations['ISSUE_DATE'].apply(
    lambda x: dob_date(x))

# cut out the approx 50 records with incoherent date format
dob_violations = dob_violations[~dob_violations['issue_date'].isnull()]

In [28]:
# deal with inconsistent datetime
def ecb_date(data):
    try:
        return datetime.datetime.strptime(str(data), '%Y%m%d')
    except:
        return float("NaN")

ecb['issue_date'] = ecb['ISSUE_DATE'].apply(
    lambda x: ecb_date(x))

# cut out the 85 records with incoherent date format
ecb = ecb[~ecb['issue_date'].isnull()]

In [29]:
# truncate dates after 12/31/2015, but keeping dates prior
dob_complaints = dob_complaints[dob_complaints.date_entered<'2016-01-01']

dob_violations = dob_violations[dob_violations.issue_date<datetime.datetime(2016,1,1,0,0)]

ecb = ecb[ecb.issue_date<datetime.datetime(2016,1,1,0,0)]

permits = permits[permits.issuance_date<datetime.datetime(2016,1,1,0,0)]

Pre-process DOB/ECB data as with the PLUTO data (group by zip and scale)

In [30]:
# same as above, group various DOB / ECB datasets to zip and scale

# groupby zip - # complaint category in zip / total complaints in zip
complaints_by_zip = scale_and_group_zip(dob_complaints,'zipcode','Complaint Category',
                                       multiple=True,header_prefix='DOB_complaint_')

# groupby zip - # disposition code in zip / total dispositions in zip
disposition_by_zip = scale_and_group_zip(dob_complaints,'zipcode','Disposition Code',
                                       multiple=True,header_prefix='DOB_dispos_',dispose=True)

# groupby zip - # dob violations type in zip / total violations in zip
violations_by_zip = scale_and_group_zip(dob_violations,'zipcode','VIOLATION_TYPE',
                                       multiple=True,header_prefix='DOB_violation_')

# groupby zip - # ecb violation type in zip / total violations in zip
ecb_violations_by_zip = scale_and_group_zip(ecb,'zipcode','VIOLATION_TYPE',
                                      multiple=True,header_prefix='ECB_violation_')

# groupby zip - # ecb violation type in zip / total violations in zip
ecb_infractions_by_zip = scale_and_group_zip(ecb,'zipcode','INFRACTION_CODE1',
                                       multiple=True,header_prefix='ECB_infraction_')

# groupby zip - # permit type in zip / total permits in zip
permit_by_zip = scale_and_group_zip(permits,'Zip Code','Permit Type',
                                       multiple=True,header_prefix='DOB_permit_')

# groupby zip - # oil or gas permits in zip / total permits in zip
oil_gas_permit_by_zip = scale_and_group_zip(permits,'Zip Code','Oil Gas',
                                       multiple=True,header_prefix='DOB_permit_',og=True)

Pre-process census data and aggregate by zipcode

In [31]:
# merge race and income
census = race.merge(income,how='outer',on='GEOID')

# add zip code column
census['TRACT'] = census.GEOID.apply(lambda x: str(x))

census_zip = census.merge(zip_tract_nyc,how='inner',on='TRACT')

census_zip.rename(columns={'ZIPCODE':'zipcode'},inplace=True)

In [32]:
# the following assorted census attributes are aggregated by zip
census_features_by_zip = pd.DataFrame()

# function to create ratio of given PLUTO category per zip code
def agg_census_zip(data,zip_col_name,oldfield,newfield,denominator='TOTAL_POPULATION'):
    census_features_by_zip[newfield] = data.groupby(
        zip_col_name)[oldfield].sum()*1.0/data.groupby(zip_col_name)[denominator].sum()

In [33]:
# same as above, group various census data to zip and scale

# groupby zip: total population in zip
census_features_by_zip['total_population'] = census_zip.groupby('zipcode')['TOTAL_POPULATION'].sum()

# groupby zip: total households in zip
census_features_by_zip['total_households'] = census_zip.groupby('zipcode')['TOTAL_HOUSEHOLDS'].sum()

# demo data by zip: race population / total population of zip
for col in census_zip.columns[3:9]:
    agg_census_zip(census_zip,'zipcode',col,col+'_ratio','TOTAL_POPULATION')

In [34]:
# aggregate income by mutliplying mean income for census tract by total households in tract
# to get total projected income for the tract
# then aggregate total income to zipcode level and divide that by total households aggregated to zip
# to get a mean income per household for the zip

census_zip['total_income'] = census_zip['MEAN_INCOME']*census_zip['TOTAL_HOUSEHOLDS']

census_features_by_zip['zip_household_mean_income'] = census_zip.groupby(
    'zipcode')['total_income'].sum()*1.0/census_zip.groupby('zipcode')['TOTAL_HOUSEHOLDS'].sum()

In [35]:
# set zipcode as column for later merge

census_features_by_zip.reset_index(inplace=True)
census_features_by_zip = census_features_by_zip.rename(columns={'zipcode':'ZipCode'}).copy()

Preprocess the NYC zipcode shapefiles

In [36]:
# remove duplicate zips (just keeping first listed)
index_list = []
for i in nyc_zips.ZIPCODE:
    temp_index = nyc_zips.ZIPCODE[nyc_zips.ZIPCODE==i].index.tolist()

    if len(temp_index)>1:
        index_list += temp_index[1:]

index_list = set(index_list)

zip_t = nyc_zips.T

zip_drop = zip_t.drop(index_list,axis=1)

nyc_zips_set = zip_drop.T

### Merge datasets

In [37]:
# merge gas
fdny_gas_zip['ZipCode'] = fdny_gas_zip['ZIP_CODE'].astype(str)
fdny_gas_zip['gas_incidents'] = fdny_gas_zip['IM_INCIDENT_KEY']
fdny_gas_zip_2 = fdny_gas_zip.drop(['ZIP_CODE','IM_INCIDENT_KEY'],axis=1)

Note, since FDNY zips are a subset of PLUTO zips, merging all PLUTO first, then performing left-join of FDNY on PLUTO

In [38]:
# merge PLUTO datasets

# age and building class
merged_pluto = avg_bldg_age_by_zip.merge(bldgclass_by_zip,how='left',on='ZipCode')

# merge land use
merged_pluto = merged_pluto.merge(landuse_by_zip,how='left',on='ZipCode')

# merge remaing PLUTO attributes
merged_pluto = merged_pluto.merge(pluto_attrib_by_zip,how='left',on='ZipCode')

# merge with FDNY
pluto_fdny = merged_pluto.merge(fdny_gas_zip_2, how = 'left',on='ZipCode')

In [39]:
# merge DOB / ECB data

# dob complaints
pluto_fdny_dob = pluto_fdny.merge(complaints_by_zip,how='left',on='ZipCode')

# dob dispositions
pluto_fdny_dob = pluto_fdny_dob.merge(disposition_by_zip,how='left',on='ZipCode')

# dob violations
pluto_fdny_dob = pluto_fdny_dob.merge(violations_by_zip,how='left',on='ZipCode')

# ecb violations
pluto_fdny_dob = pluto_fdny_dob.merge(ecb_violations_by_zip,how='left',on='ZipCode')

# ecb infractions
pluto_fdny_dob = pluto_fdny_dob.merge(ecb_infractions_by_zip,how='left',on='ZipCode')

# dob permit type
pluto_fdny_dob = pluto_fdny_dob.merge(permit_by_zip,how='left',on='ZipCode')

# dob oil or gas permit
pluto_fdny_dob = pluto_fdny_dob.merge(oil_gas_permit_by_zip,how='left',on='ZipCode')

In [40]:
# merge census data
pluto_fdny_dob_census = pluto_fdny_dob.merge(census_features_by_zip,how='left',on='ZipCode')

In [48]:
# merge zip shapefiles
nyc_zips_set['ZipCode'] = nyc_zips_set['ZIPCODE'].astype(str)
nyc_zips_clean = nyc_zips_set[['ZipCode','geometry','AREA']].copy()

master_merged = pluto_fdny_dob_census.merge(nyc_zips_clean,how='left',on='ZipCode')

# move target (i.e. dependent) variable to last column and rename "total_gas_incidents"
master_merged['total_gas_incidents'] = master_merged['gas_incidents']
del master_merged['gas_incidents']

# add column gas incidents per building unit
master_merged['gas_incidents_per_bldg_unit'] = master_merged['total_gas_incidents']*1.0/master_merged['total_units']

In [49]:
master_merged.to_csv('processed_data/pluto_fdny_dob_census_to_zipcode.csv',index=False)

### OUTPUT: final merged 

#### FDNY
- Merge with zip code shapefile

#### PLUTO features
- Avg building age per zipcode
- Ratio of each building class per zip code
- Ratio of each land use per zip code
- Building use ratio (commercial, residential, office, retail) per zip code
- Residential unit density per zip code
- Ave Unit area per zip code
- Value per ft per zip code
- Total units per zip code

#### DOB/ECB features
- Ratio of each DOB complaint type per zip code
- Ratio of each DOB complaint disposition per zip code
- Ratio of each DOB violation type per zip code
- Ratio of each ECB violation type per zip code
- Ratio of each DOB work permit type per zip code
- Ratio of oil or gas permits out of all permits per zip code

#### Census data features
- Total population per zip code
- Total households per zip code
- Mean income (calculated by: mean income per census tract * total households in ct to get total income per ct, 
    then aggregate total income to zip code and divide by total households aggregated to zip code
- Ratio of various racial groups out of total population per zip code

#### Target variables (last two columns in the output dataset)
- Total gas incidents per zip code
- Ratio of gas incidents per total building units in zip code